In [2]:
# import libaries
import os
import requests
from bs4 import BeautifulSoup as bs
import re
import time

In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

class ScrapingFrancetravail:
    # En-têtes pour imiter un vrai navigateur
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    def __init__(self):
        self.base_url = "https://www.francetravail.fr"  # Base URL for France Travail

    def scrape(self, url):
        """
        Scrape data from a single France Travail URL.
        
        :param url: The URL to scrape.
        :return: Scraped data as a dictionary or None if an error occurs.
        """
        try:
            # Make a request to the URL
            print(f"Scraping France Travail URL: {url}")
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()  # Raise an error for bad status codes

            # Parse the HTML content
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract relevant data
            job_title = soup.find("h1").text.strip() if soup.find("h1") else "No Title"
            job_description = soup.find("div", class_="description").text.strip() if soup.find("div", class_="description") else "No Description"
            company = soup.find("span", class_="company-name").text.strip() if soup.find("span", class_="company-name") else "No Company"
            location = soup.find("span", class_="job-location").text.strip() if soup.find("span", class_="job-location") else "No Location"

            return {
                "title": job_title,
                "company": company,
                "location": location,
                "description": job_description,
                "url": url
            }
        except requests.RequestException as e:
            print(f"Error scraping {url}: {e}")
            return None

    def scrapMany(self, keyWord, max_results=10):
        """
        Scrape multiple job listings based on a keyword.
        
        :param keyWord: The keyword to search for.
        :param max_results: The maximum number of results to return.
        :return: A list of scraped job listings.
        """
        try:
            # Construct the search URL
            search_url = f"{self.base_url}/recherche?query={keyWord}"
            print(f"Searching France Travail for keyword: {keyWord}")
            response = requests.get(search_url, headers=self.headers)
            response.raise_for_status()

            # Parse the search results
            soup = BeautifulSoup(response.text, "html.parser")
            job_listings = []

            # Find job listing elements
            job_elements = soup.find_all("div", class_="job-listing", limit=max_results)
            for job in job_elements:
                # Extract job link
                job_link = job.find("a")["href"]
                full_job_url = urljoin(self.base_url, job_link)

                # Scrape individual job details
                job_data = self.scrape(full_job_url)
                if job_data:
                    job_listings.append(job_data)

            return job_listings
        except requests.RequestException as e:
            print(f"Error searching for jobs: {e}")
            return []

    def get_access_token(self):
        """
        Retrieve the access token for the Pôle Emploi API.
        
        :return: The access token.
        """
        payload = {
            "grant_type": "client_credentials",
            "client_id": CLIENT_ID,
            "client_secret": CLIENT_SECRET,
            "scope": "api_offresdemploiv2 o2dsoffre"
        }
        try:
            response = requests.post(TOKEN_URL, data=payload)
            response.raise_for_status()
            return response.json()["access_token"]
        except requests.RequestException as e:
            print(f"Error retrieving access token: {e}")
            return None

    def get_job_offers(self, access_token, keyword, location=None, limit=5):
        """
        Fetch job offers from the Pôle Emploi API.
        
        :param access_token: The access token for the API.
        :param keyword: The keyword to search for.
        :param location: The postal code or location (optional).
        :param limit: The maximum number of results to return.
        :return: A list of job offers.
        """
        url = "https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search"
        headers = {"Authorization": f"Bearer {access_token}"}
        params = {
            "motsCles": keyword,
            "lieuTravail.codePostal": location,
            "range": f"0-{limit-1}"  # Number of offers to retrieve
        }

        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            return response.json().get("resultats", [])
        except requests.RequestException as e:
            print(f"Error fetching job offers: {e}")
            return []

    def display_offers(self, offers):
        """
        Display job offers in a readable format.
        
        :param offers: A list of job offers.
        """
        for offer in offers:
            titre = offer.get("intitule", "N/A")
            entreprise = offer.get("entreprise", {}).get("nom", "N/A")
            contrat = offer.get("typeContrat", "N/A")
            localisation = offer.get("lieuTravail", {}).get("libelle", "N/A")
            description = offer.get("description", "N/A")

            print("=" * 50)
            print(f"Titre : {titre}")
            print(f"Entreprise : {entreprise}")
            print(f"Contrat : {contrat}")
            print(f"Localisation : {localisation}")
            print(f"Description :\n{description}")

# Exécution du script
if __name__ == "__main__":
    # Create an instance of ScrapingFrancetravail
    scraper = ScrapingFrancetravail()

    # Option 1: Scrape job listings from the website
    keyword = "Data Analyst"
    job_listings = scraper.scrapMany(keyWord=keyword, max_results=5)
    for job in job_listings:
        print("=" * 50)
        print(f"Title: {job['title']}")
        print(f"Company: {job['company']}")
        print(f"Location: {job['location']}")
        print(f"Description: {job['description']}")
        print(f"URL: {job['url']}")

    # Option 2: Fetch job listings from the Pôle Emploi API
    access_token = scraper.get_access_token()
    if access_token:
        keyword = "Data *"
        location = "75000"  # Code postal (optionnel)
        offers = scraper.get_job_offers(access_token, keyword, location, limit=10)
        scraper.display_offers(offers)

Searching France Travail for keyword: Data Analyst
Error searching for jobs: 404 Client Error: Not Found for url: https://www.francetravail.fr/recherche?query=Data%20Analyst
Titre : Data engineer (H/F)
Entreprise : IONE TALENTS & TECHNOLOGY
Contrat : CDI
Localisation : 92 - CHATILLON
Description :
IONE Talents & Technology, société de Conseil en Système d'Information avec une forte expertise métier créée par un expert dans le domaine de la data, IONE Talents & Technology c'est surtout un ensemble de consultant.e.s capables d'intervenir à tous les niveaux d'un Système d'Information.

Si vous souhaitez rejoindre une entreprise à taille humaine où chaque employé peut avoir un réel rôle à jouer dans son développement, vous êtes au bon endroit.

Dans le cadre de notre croissance sur la région d'ile de France, nous recrutons actuellement un(e) Data engineer IT en CDI pour pour intervenir chez nos clients dans la mise en place de nouveaux projets.


Profil recherché : 
- Expérience significat

In [3]:
# information about the job
url = "https://candidat.francetravail.fr/offres/recherche/detail/189KKQR" 

In [4]:
# En-têtes pour imiter un vrai navigateur
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Envoyer la requête HTTP
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = bs(response.text, "html.parser")

    # Extraction du titre de l'offre
    titre = soup.find("h1").text.strip() if soup.find("h1") else "Non disponible"

    # Extraction du nom de l'entreprise
    entreprise = soup.find("div", class_="t4").text.strip() if soup.find("div", class_="t4") else "Non renseigné"

    # Extraction de la description du poste
    description = soup.find("div", class_="description").text.strip() if soup.find("div", class_="description") else "Non disponible"

    # Extraction du type de contrat
    contrat = soup.find("span", class_="contrat").text.strip() if soup.find("span", class_="contrat") else "Non renseigné"

    # Extraction de la localisation
    localisation = soup.find("span", class_="lieu").text.strip() if soup.find("span", class_="lieu") else "Non renseigné"

    # Affichage des résultats
    print(f"Titre : {titre}")
    print(f"Entreprise : {entreprise}")
    print(f"Contrat : {contrat}")
    print(f"Localisation : {localisation}")
    print(f"Description :\n{description}")

else:
    print(f"Échec de la récupération des données. Code HTTP : {response.status_code}")

Titre : Offre n° 189KKQRDéveloppeur Full Stack .Net  (H/F)
Entreprise : Non renseigné
Contrat : Non renseigné
Localisation : Non renseigné
Description :
Rejoignez une équipe sympathique et dynamique, dans un groupe en forte croissance !

POSTE et MISSION

Au sein du pôle «Développement et R&D», vous serez intégré-e à l'équipe chargée du développement de nos Suites logicielles de gestion des risques Qualité, Sécurité, Environnement. A ce titre, vous travaillerez sur la conception et le développement de nos logiciels (algorithmes, IHM, API, persistance, Front End, Back End, etc) :

- Intégrez une petite équipe d'ingénieurs, stable, qui travaille en étroite collaboration avec les autres services (notamment métier), selon la méthode Agile (Scrum), sur des projets durables,

- A partir de spécifications fonctionnelles, vous concevez et développez les évolutions et nouvelles fonctionnalités de nos applications web en ASP.Net MVC (C#, Javascript, frontEnd) et de notre moteur métier (en C#, C+

In [7]:
import requests

# Paramètres d'authentification (remplace avec tes propres identifiants)
CLIENT_ID = "PAR_sise_6ecb7de16380e8802b4565918fcd2dab184e5916e58cba3b4ba4e09a9d124064"
CLIENT_SECRET = "fdd7ac75d36925e84f2fcbe0ed7cecf9619d01553708c4ac58f2c648e6bf3927"
TOKEN_URL = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"

# Récupération du token d'accès
def get_access_token():
    payload = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "api_offresdemploiv2 o2dsoffre"
    }
    response = requests.post(TOKEN_URL, data=payload)
    response.raise_for_status()
    return response.json()["access_token"]

# Récupération des offres d'emploi
def get_job_offers(access_token, keyword, location=None, limit=5):
    url = "https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    params = {
        "motsCles": keyword,
        "lieuTravail.codePostal": location,
        "range": f"0-{limit-1}"  # Nombre d'offres à récupérer
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    return response.json().get("resultats", [])

# Affichage des offres
def display_offers(offers):
    for offer in offers:
        titre = offer.get("intitule", "N/A")
        entreprise = offer.get("entreprise", {}).get("nom", "N/A")
        contrat = offer.get("typeContrat", "N/A")
        localisation = offer.get("lieuTravail", {}).get("libelle", "N/A")
        description = offer.get("description", "N/A")

        print("=" * 50)
        print(f"Titre : {titre}")
        print(f"Entreprise : {entreprise}")
        print(f"Contrat : {contrat}")
        print(f"Localisation : {localisation}")
        print(f"Description :\n{description}")

In [8]:
# Exécution du script
if __name__ == "__main__":
    access_token = get_access_token()
    keyword = "Data *"  # Change selon le type de poste recherché
    location = "75000"  # Code postal (optionnel)

    offers = get_job_offers(access_token, keyword, location, limit=5)
    display_offers(offers)

Titre : DATA ARCHITECT (H/F)
Entreprise : HARDIS GROUPE
Contrat : CDI
Localisation : 38 - Grenoble
Description :
Vous possédez une solide expertise dans les technologies de la Data, l'ingénierie et la modélisation des données. Votre grande curiosité technologique et votre capacité avérée à vous adapter rapidement à toutes les évolutions font de vous un professionnel désireux de contribuer au développement d'une nouvelle activité stratégique au sein d'une entreprise axée sur l'innovation et l'ambition. Rejoignez Hardis Group ! 


Au sein de notre entité Services Tech Solutions Grenoble, nous renforçons continuellement nos équipes pour consolider notre savoir-faire en matière de Data. Dans le cadre de l'expansion de notre activité Services, Hardis Group recherche activement un Architecte Data (H/F) pour renforcer nos équipes grenobloises. 

En qualité d'expert, vous interviendrez sur diverses missions telles que le conseil, l'expertise, les audits techniques, la cartographie des processu